<a href="https://colab.research.google.com/github/Reemaalt/Detection-of-Hallucination-in-Arabic/blob/main/AnswerClusterer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



1.   load generate answers
2.   Entailment model load //trin
3.   Clustering useing the ent model
4.   Save the output into a file





# set ups

In [ ]:
!huggingface-cli login
!pip install datasets
!pip install  transformers torch evaluate


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `week1 test` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `week1 t

In [ ]:
import torch
import json
import pandas as pd
import os
import torch.nn.functional as F
from transformers import (
    ElectraForSequenceClassification,
    ElectraTokenizerFast,
    Trainer,
    TrainingArguments,
    pipeline,
    AutoTokenizer,
    AutoModelForSequenceClassification
)
from datasets import load_dataset
import numpy as np
from evaluate import load
import json
from google.colab import files

# load and fine-tuned the  model

In [ ]:
#load functions

# Load XNLI dataset
print("Loading XNLI dataset...")
dataset = load_dataset("xnli", "ar")

# Load AraELECTRA model and tokenizer
model_name = "aubmindlab/araelectra-base-discriminator"
tokenizer = ElectraTokenizerFast.from_pretrained(model_name)
model = ElectraForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3  # entailment, neutral, contradiction
)



Loading XNLI dataset...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/58.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/392k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.64M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/503 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at aubmindlab/araelectra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#trinning functions
# Preprocess function with labels
def preprocess_function(examples):

    # Tokenize the premise and hypothesis pairs
    tokenized = tokenizer(
        examples["premise"],
        examples["hypothesis"],
        truncation=True,
        max_length=128,
        padding="max_length"
    )

    # Add labels to the tokenized output
    tokenized["labels"] = examples["label"]

    return tokenized

# Tokenize datasets
print("Tokenizing datasets...")
tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names
)

# Print sample to verify data format
print("\nSample preprocessed data:")
print(tokenized_datasets["train"][0])

# Load accuracy metric
metric = load("accuracy")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./araelectra-nli-finetuned",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="wandb",           # Add WandB reporting
    logging_dir="./logs",
    logging_steps=500,
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,         # Add tokenizer to trainer
)


# Fine-tune the model
print("\nStarting training...")
train_result = trainer.train()

# Print training results
print("\nTraining results:")
print(train_result)

# Save the fine-tuned model
model_save_path = "./araelectra-nli-finetuned"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)
print(f"\nModel saved to {model_save_path}")



Tokenizing datasets...


Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Map:   0%|          | 0/5010 [00:00<?, ? examples/s]

Map:   0%|          | 0/2490 [00:00<?, ? examples/s]


Sample preprocessed data:
{'input_ids': [2, 19, 139, 602, 298, 7249, 13782, 300, 11936, 59778, 19, 8222, 43907, 20, 3, 8222, 43907, 583, 394, 3546, 7249, 38749, 192, 700, 20, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: altwujaeire-reema (altwujaeire-reema-king-saud-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.553500,0.585348,0.776305
2,0.429200,0.629222,0.776707
3,0.322600,0.745767,0.775904



Training results:
TrainOutput(global_step=73632, training_loss=0.45865262109059346, metrics={'train_runtime': 14079.3863, 'train_samples_per_second': 83.676, 'train_steps_per_second': 5.23, 'total_flos': 7.74938740264658e+16, 'train_loss': 0.45865262109059346, 'epoch': 3.0})

Model saved to ./araelectra-nli-finetuned


In [ ]:
#Call the model
model = ElectraForSequenceClassification.from_pretrained("./araelectra-nli-finetuned")
tokenizer = ElectraTokenizerFast.from_pretrained("./araelectra-nli-finetuned")


In [ ]:
# Test the fine-tuned model
def test_model(premise, hypothesis):
    inputs = tokenizer(
        premise,
        hypothesis,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_class = torch.argmax(predictions, dim=-1).item()

    label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}
    return label_map[predicted_class], predictions[0].tolist()

# Test examples
test_examples = [
    ("القاهره عاصمه مصر", "القاهره هي عاصمة دولة مصر"),
    ("القاهره عاصمه مصر", "القاهره ليست عاصمة مصر"),
    ("القاهره عاصمه مصر", "مصر دولة عربية")
]

print("\nTesting fine-tuned model:")
for premise, hypothesis in test_examples:
    prediction, probabilities = test_model(premise, hypothesis)
    print(f"\nPremise: {premise}")
    print(f"Hypothesis: {hypothesis}")
    print(f"Prediction: {prediction}")
    print(f"Probabilities: entailment={probabilities[0]:.3f}, neutral={probabilities[1]:.3f}, contradiction={probabilities[2]:.3f}")


Testing fine-tuned model:

Premise: القاهره عاصمه مصر
Hypothesis: القاهره هي عاصمة دولة مصر
Prediction: entailment
Probabilities: entailment=0.966, neutral=0.031, contradiction=0.003

Premise: القاهره عاصمه مصر
Hypothesis: القاهره ليست عاصمة مصر
Prediction: contradiction
Probabilities: entailment=0.002, neutral=0.010, contradiction=0.988

Premise: القاهره عاصمه مصر
Hypothesis: مصر دولة عربية
Prediction: neutral
Probabilities: entailment=0.071, neutral=0.837, contradiction=0.092


# load  the generation file

In [ ]:
#load generation file
file_path = '/content/generated_answers_Llama3.1-xquadALLdataset-LOG.json'

with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

df = pd.DataFrame(data)
print(df.head())

  question_id                                           question  \
0        None  ما هي إحدى الطرق التي يمكن بها تشفير الرسوم ال...   
1        None  ما الطرفان الحاكمان اللذان يملكان الحق الرفض ف...   
2        None            من لديه صلاحية تنقيح الدستور الفيكتوري؟   

                                     original_answer  \
0  {'text': ['المصفوفات المجاورة'], 'answer_start...   
1  {'text': ['البرلمان الأوربي ومجلس الاتحاد الأو...   
2  {'text': ['برلمان فيكتوريا'], 'answer_start': ...   

                                   generated_answers  \
0  [يمكن تشفير الرسوم البيانية عن طريق إجراء عملي...   
1  [الطرفان الحاكمان اللذان يملكان الحق الرفض فيم...   
2  [It was written to protect the rights of Engli...   

                             avg_neg_log_likelihoods  
0  [12.670990943908691, 11.9406099319458, 1.15451...  
1  [9.86954402923584, 2.5988807678222656, 9.86954...  
2  [2.1815671920776367, 1.377522587776184, 19.755...  


# entailment functions

In [ ]:
# Define the entailment checking function BEFORE using it
def check_entailment_arabic(sentence1, sentence2):
    # Load the fine-tuned model

    model_path = "./araelectra-nli-finetuned"
    model = ElectraForSequenceClassification.from_pretrained(model_path)
    tokenizer = ElectraTokenizerFast.from_pretrained(model_path)

    # Move to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # Prepare input
    inputs = tokenizer(
        sentence1,
        sentence2,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

    # Move inputs to device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Get prediction
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = F.softmax(logits, dim=1)
        predicted_class = torch.argmax(probs, dim=1).item()

    # Map prediction to label
    return "entailment" if predicted_class == 0 else "neutral" if predicted_class == 1 else "contradiction"

def bidirectional_entailment(question, answer1, answer2):
    entailment1 = check_entailment_arabic(answer1, answer2)
    entailment2 = check_entailment_arabic(answer2, answer1)
    return entailment1 == "entailment" and entailment2 == "entailment"

# Clusters generated answers for a specific question based on bidirectional entailment.
def process_question(question_id, generated_answers):
    clusters = []  # List of clusters

   # Iterate over all generated answers
    for answer in generated_answers:
        added_to_cluster = False # Track whether the answer is added to a cluster

      # Compare the answer with representatives of existing clusters
        for cluster in clusters:
            representative = cluster[0] # Take the first answer as representative
            if bidirectional_entailment(question_id, representative, answer):
                cluster.append(answer) # Add the answer to the cluster
                added_to_cluster = True
                break
   # If the answer does not fit into any cluster, create a new cluster.
        if not added_to_cluster:
            clusters.append([answer])  # Create a new cluster
    return clusters


# Processes the dataset for entailment clustering
def process_dataset(data):
    results = {}

    for question in data:
        question_id = question["question_id"]
        question_text = question["question"]
        original_answer = question["original_answer"]
        generated_answers = question["generated_answers"]
        total_avg_neg_log_likelihoods_for_clusters = question["avg_neg_log_likelihoods"]

        print(f"\nProcessing question {question_id}: {question_text}")

        # Generate clusters
        clusters = process_question(question_id, generated_answers)

        results[question_id] = {
            "question_id": question_id,
            "question": question_text,
            "original_answer": original_answer,
            "clusters": clusters,
            "total_avg_neg_log_likelihoods_for_clusters": total_avg_neg_log_likelihoods_for_clusters
        }

    return results



#  Apply the clustering and save the clusters file

In [ ]:
# Step 3: Clustering
# Apply the clustering to the loaded dataset
print("\nApplying clustering to the loaded dataset...")


entailment_clusters = process_dataset(data)

# Save results to JSON file
output_file = "entailment_clusters_Llama3.1-xquadAll-data.json"

with open(output_file, 'w', encoding='utf-8') as file:
    json.dump(entailment_clusters, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file}")

files.download(output_file)



Applying clustering to the loaded dataset...

Processing question None: ما هي إحدى الطرق التي يمكن بها تشفير الرسوم البيانية؟

Processing question None: ما الطرفان الحاكمان اللذان يملكان الحق الرفض فيما يخص سن القوانين؟

Processing question None: من لديه صلاحية تنقيح الدستور الفيكتوري؟
Results saved to entailment_clusters_Llama3.1-xquadAll-data.json
Results saved to entailment_clusters_Llama3.1-xquadAll-data.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>